# RAG from scratch

- Adapted from https://github.com/opendatahub-io/llama-stack-demos/
- Requires ollama template 

# Variables

The variables are configured related to the distribution we are connected

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
base_url=os.getenv("BASE_URL") #local "http://localhost:8321"
chunk_size_in_tokens=os.getenv("CHUNK_SIZE_TOKEN") #512
collection_name=os.getenv('COLLECTION_NAME')  ##all-MiniLM-L6-v2"
embedding_model_name=os.getenv('EMBEDDING_MODEL_NAME')  
embedding_size=int(os.getenv('EMBEDDING_SIZE'))   
model_name=os.getenv("MODEL_NAME") 

### Define client and test connection to server

In [2]:
from llama_stack_client import LlamaStackClient
client = LlamaStackClient(
    base_url=base_url,
)

In [3]:
client.models.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', type='model', provider_resource_id='all-minilm:latest', model_type='embedding'),
 Model(identifier='llama3.2:3b-instruct-fp16', metadata={}, api_model_type='llm', provider_id='ollama', type='model', provider_resource_id='llama3.2:3b-instruct-fp16', model_type='llm')]

### Ingest data to vector db 

In [34]:
from llama_stack_client import RAGDocument

#urls = ["chat.rst", "llama3.rst", "memory_optimizations.rst", "lora_finetune.rst"]
urls = ["chat.rst", "llama3.rst"]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

In [35]:
vector_db_id = collection_name
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=embedding_model_name,
    embedding_dimension=embedding_size,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-dbs "HTTP/1.1 200 OK"


VectorDBRegisterResponse(embedding_dimension=384, embedding_model='all-MiniLM-L6-v2', identifier='demoInnovate-vectordb', provider_id='faiss', type='vector_db', provider_resource_id='demoInnovate-vectordb', owner={'principal': '', 'attributes': {}})

Ingesting documents into a vector database:

In [11]:
import rich
rich.print(documents)

[
    {
        'document_id': 'num-0',
        'content': 'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/chat.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    },
    {
        'document_id': 'num-1',
        'content': 'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/llama3.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    },
    {
        'document_id': 'num-2',
        'content': 
'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/memory_optimizations.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    },
    {
        'document_id': 'num-3',
        'content': 
'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/lora_finetune.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    }
]

In [12]:
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=chunk_size_in_tokens,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 200 OK"


## Testing vector db 

In [ ]:
prompt = "What are the top 5 topics that were explained? Only list succinct bullet points."

### Using tool_runtime api that can query in multiple collection

In [ ]:
# higher level tool provides packaged results, can span multiple dbs
tool_response = client.tool_runtime.rag_tool.query(
    content=prompt, vector_db_ids=[vector_db_id]
)
rich.print(tool_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/query "HTTP/1.1 200 OK"


QueryResult(
    metadata={'document_ids': ['num-0', 'num-0', 'num-0', 'num-3', 'num-3']},
    content=[
        TextContentItem(
            text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
            type='text'
        ),
        TextContentItem(
            text='Result 1\nContent:  Instruct.\n\n.. _prompt_template_vs_special_tokens:\n\nTokenizing prompt 
templates & special tokens\n--------------------------------------------\n\nLet\'s say I have a sample of a single 
user-assistant turn accompanied with a system\nprompt:\n\n.. code-block:: python\n\n    sample = [\n        {\n    
"role": "system",\n            "content": "You are a helpful, respectful, and honest assistant.",\n        },\n    
{\n            "role": "user",\n            "content": "Who are the most influential hip-hop artists of all 
time?",\n        },\n        {\n            "role": "assistant",\n            "content": "Here is a list of some of
the most influential hip-hop "\n            "artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.",\n    
},\n    ]\n\nNow, let\'s format this with the :class:`~torchtune.models.llama2.Llama2ChatTemplate` class and\nsee 
how it gets tokenized. The Llama2ChatTemplate is an example of a **prompt template**,\nwhich simply structures a 
prompt with flavor text to indicate a certain task.\n\n.. code-block:: python\n\n    from torchtune.data import 
Llama2ChatTemplate, Message\n\n    messages = [Message.from_dict(msg) for msg in sample]\n    formatted_messages = 
Llama2ChatTemplate.format(messages)\n    print(formatted_messages)\n    # [\n    #     Message(\n    #         
role=\'user\',\n    #         content=\'[INST] <<SYS>>\\nYou are a helpful, respectful, and honest 
assistant.\\n<</SYS>>\\n\\nWho are the most influential hip-hop artists of all time? [/INST] \',\n    #         
...,\n    #     ),\n    #     Message(\n    #         role=\'assistant\',\n    #         content=\'Here is a list 
of some of the most influential hip-hop artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.\',\n    #   
...,\n    #     ),\n    # ]\n\nThere are also special tokens used by Llama2, which are not in the prompt 
template.\nIf you look at our :class:`~torchtune.models.llama2.Llama2ChatTemplate` class, you\'ll notice that\nwe 
don\'t include the :code:`<s>` and :code:`</s>` tokens. These are the beginning-of-sequence\n(BOS) and 
end-of-sequence (EOS) tokens that are represented differently\nMetadata: {\'document_id\': \'num-0\'}\n',
            type='text'
        ),
        TextContentItem(
            text='Result 2\nContent:  Instruct.\n\n.. _prompt_template_vs_special_tokens:\n\nTokenizing prompt 
templates & special tokens\n--------------------------------------------\n\nLet\'s say I have a sample of a single 
user-assistant turn accompanied with a system\nprompt:\n\n.. code-block:: python\n\n    sample = [\n        {\n    
"role": "system",\n            "content": "You are a helpful, respectful, and honest assistant.",\n        },\n    
{\n            "role": "user",\n            "content": "Who are the most influential hip-hop artists of all 
time?",\n        },\n        {\n            "role": "assistant",\n            "content": "Here is a list of some of
the most influential hip-hop "\n            "artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.",\n    
},\n    ]\n\nNow, let\'s format this with the :class:`~torchtune.models.llama2.Llama2ChatTemplate` class and\nsee 
how it gets tokenized. The Llama2ChatTemplate is an example of a **prompt template**,\nwhich simply structures a 
prompt with flavor text to indicate a certain task.\n\n.. code-block:: python\n\n    from torchtune.data import 
Llama2ChatTemplate, Message\n\n    messages = [Message.from_dict(msg) for msg in sample]\n    formatted_messages = 
Llama2ChatTemplate.format(messages)\n    print(formatted_messages)\n    # [\n    #     Message(\n    #         
role=\'user\',\n    #         co

In [33]:
help(client.tool_runtime.rag_tool.query)

Help on method query in module llama_stack_client.resources.tool_runtime.rag_tool:

query(*, content: 'InterleavedContent', vector_db_ids: 'List[str]', query_config: 'QueryConfig | NotGiven' = NOT_GIVEN, extra_headers: 'Headers | None' = None, extra_query: 'Query | None' = None, extra_body: 'Body | None' = None, timeout: 'float | httpx.Timeout | None | NotGiven' = NOT_GIVEN) -> 'QueryResult' method of llama_stack_client.resources.tool_runtime.rag_tool.RagToolResource instance
    Query the RAG system for context; typically invoked by the agent
    
    Args:
      content: A image content item
    
      query_config: Configuration for the RAG query generation.
    
      extra_headers: Send extra headers
    
      extra_query: Add additional query parameters to the request
    
      extra_body: Add additional JSON properties to the request
    
      timeout: Override the client-level default timeout for this request, in seconds



### Query the vector io collection directly

In [18]:
# we can also query the vector db directly
db_response = client.vector_io.query(
    vector_db_id=vector_db_id,
    query=prompt,
)
rich.print(db_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-io/query "HTTP/1.1 200 OK"


QueryChunksResponse(
    chunks=[
        Chunk(
            content=',\n    and (b) the memory constraints of your hardware.\n\nThe preceding command will run a 
LoRA finetune with torchtune\'s factory settings, but we may want to experiment a bit.\nLet\'s take a closer look 
at some of the :code:`lora_finetune_distributed` config.\n\n.. code-block:: yaml\n\n  # Model Arguments\n  model:\n
_component_: lora_llama2_7b\n    lora_attn_modules: [\'q_proj\', \'v_proj\']\n    lora_rank: 8\n    lora_alpha: 
16\n  ...\n\nWe see that the default is to apply LoRA to Q and V projections with a rank of 8.\nSome experiments 
with LoRA have found that it can be beneficial to apply LoRA to all linear layers in\nthe self-attention, and to 
increase the rank to 16 or 32. Note that this is likely to increase our max memory,\nbut as long as we keep 
:code:`rank<<embed_dim`, the impact should be relatively minor.\n\nLet\'s run this experiment. We can also increase
alpha (in general it is good practice to scale alpha and rank together).\n\n.. code-block:: bash\n\n    tune run 
--nnodes 1 --nproc_per_node 2 lora_finetune_distributed --config llama2/7B_lora \\\n    
lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    lora_rank=32 lora_alpha=64 
output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between this run and our baseline over
the first 500 steps can be seen below.\n\n.. image:: /_static/img/lora_experiment_loss_curves.png\n\n.. note::\n   
The above figure was generated with W&B. You can use torchtune\'s 
:class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you will need to 
install W&B and setup an account separately. For more details on\n    using W&B in torchtune, see our 
":ref:`wandb_logging`" recipe.\n\n.. _lora_tutorial_memory_tradeoff_label:\n\nTrading off memory and model 
performance with LoRA\n--------------------------------------------------\n\nIn the preceding example, we ran LoRA 
on two devices. But given LoRA\'s low memory footprint, we can run fine-tuning\non a single device using most 
commodity GPUs which support `bfloat16 <https://',
            metadata={'document_id': 'num-3', 'token_count': 512.0, 'metadata_token_count': 1.0},
            embedding=[
                0.00406104,
                -0.048808675,
                -0.054821957,
                -0.05163342,
                0.009840118,
                0.09178936,
                -0.0101559,
                0.04716209,
                0.02568053,
                -0.014743081,
                -0.023843573,
                0.03369005,
                -0.018293295,
                0.022487223,
                -0.010905134,
                0.07937251,
                0.108631425,
                0.018547243,
                -0.056368362,
                0.012521768,
                0.06545227,
                -0.07763688,
                0.10424298,
                0.017161276,
                0.06960253,
                -0.00437907,
                -0.008911352,
                -0.0038266839,
                0.039670363,
                -0.12211394,
                0.06891109,
                0.06289771,
                0.07961002,
                0.04354608,
                -0.07493716,
                0.008346766,
                -0.05844109,
                -0.034176067,
                -0.06328286,
                0.031867906,
                0.066620894,
                0.060622793,
                -0.068750225,
                0.014350068,
                0.09729832,
                -0.034628067,
                0.023585148,
                -0.06224204,
                -0.023213899,
                -0.030940313,
                -0.05188855,
                -0.009149888,
                -0.03583717,
                0.045438588,
                -0.09836591,
                -0.053864855,
                -0.03148368,
                -0.006111429,
                0.021

## Creating context joining all the search

In [20]:
# prompt_context = tool_response.content
prompt_context = "\n".join([c.content for c in db_response.chunks])

In [21]:
messages = [{"role": "system", "content": "You are a helpful assistant."}]
extended_prompt = f"""
Please answer the given query using the context below.

QUERY:
{prompt}

CONTEXT:
{prompt_context}
"""
messages.append({"role": "user", "content": extended_prompt})
rich.print(messages)

[
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {
        'role': 'user',
        'content': '\nPlease answer the given query using the context below.\n\nQUERY:\nWhat are the top 5 topics 
that were explained? Only list succinct bullet points.\n\nCONTEXT:\n,\n    and (b) the memory constraints of your 
hardware.\n\nThe preceding command will run a LoRA finetune with torchtune\'s factory settings, but we may want to 
experiment a bit.\nLet\'s take a closer look at some of the :code:`lora_finetune_distributed` config.\n\n.. 
code-block:: yaml\n\n  # Model Arguments\n  model:\n    _component_: lora_llama2_7b\n    lora_attn_modules: 
[\'q_proj\', \'v_proj\']\n    lora_rank: 8\n    lora_alpha: 16\n  ...\n\nWe see that the default is to apply LoRA 
to Q and V projections with a rank of 8.\nSome experiments with LoRA have found that it can be beneficial to apply 
LoRA to all linear layers in\nthe self-attention, and to increase the rank to 16 or 32. Note that this is likely to
increase our max memory,\nbut as long as we keep :code:`rank<<embed_dim`, the impact should be relatively 
minor.\n\nLet\'s run this experiment. We can also increase alpha (in general it is good practice to scale alpha and
rank together).\n\n.. code-block:: bash\n\n    tune run --nnodes 1 --nproc_per_node 2 lora_finetune_distributed 
--config llama2/7B_lora \\\n    lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    
lora_rank=32 lora_alpha=64 output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between 
this run and our baseline over the first 500 steps can be seen below.\n\n.. image:: 
/_static/img/lora_experiment_loss_curves.png\n\n.. note::\n    The above figure was generated with W&B. You can use
torchtune\'s :class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you 
will need to install W&B and setup an account separately. For more details on\n    using W&B in torchtune, see our 
":ref:`wandb_logging`" recipe.\n\n.. _lora_tutorial_memory_tradeoff_label:\n\nTrading off memory and model 
performance with LoRA\n--------------------------------------------------\n\nIn the preceding example, we ran LoRA 
on two devices. But given LoRA\'s low memory footprint, we can run fine-tuning\non a single device using most 
commodity GPUs which support `bfloat16 <https://\n,\n    and (b) the memory constraints of your hardware.\n\nThe 
preceding command will run a LoRA finetune with torchtune\'s factory settings, but we may want to experiment a 
bit.\nLet\'s take a closer look at some of the :code:`lora_finetune_distributed` config.\n\n.. code-block:: 
yaml\n\n  # Model Arguments\n  model:\n    _component_: lora_llama2_7b\n    lora_attn_modules: [\'q_proj\', 
\'v_proj\']\n    lora_rank: 8\n    lora_alpha: 16\n  ...\n\nWe see that the default is to apply LoRA to Q and V 
projections with a rank of 8.\nSome experiments with LoRA have found that it can be beneficial to apply LoRA to all
linear layers in\nthe self-attention, and to increase the rank to 16 or 32. Note that this is likely to increase 
our max memory,\nbut as long as we keep :code:`rank<<embed_dim`, the impact should be relatively minor.\n\nLet\'s 
run this experiment. We can also increase alpha (in general it is good practice to scale alpha and rank 
together).\n\n.. code-block:: bash\n\n    tune run --nnodes 1 --nproc_per_node 2 lora_finetune_distributed --config
llama2/7B_lora \\\n    lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    lora_rank=32 
lora_alpha=64 output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between this run and our
baseline over the first 500 steps can be seen below.\n\n.. image:: 
/_static/img/lora_experiment_loss_curves.png\n\n.. note::\n    The above figure was generated with W&B. You can use
torchtune\'s :class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you 
will need 

In [23]:
response = client.inference.chat_completion(
    messages=messages,
    model_id=model_name,
    timeout=600
)
rich.print(response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/inference/chat-completion "HTTP/1.1 200 OK"


ChatCompletionResponse(
    completion_message=CompletionMessage(
        content='Here are the top 5 topics that were explained:\n\n* LoRA (Low-Rank Adaptation)\n* Memory 
constraints of hardware\n* Fine-tuning with LoRA on a single device\n* Experimenting with different LoRA 
configurations\n* Trading off memory and model performance with LoRA',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    logprobs=None,
    metrics=[
        Metric(metric='prompt_tokens', value=1592.0, unit=None),
        Metric(metric='completion_tokens', value=68.0, unit=None),
        Metric(metric='total_tokens', value=1660.0, unit=None)
    ]
)

### Using Agent 

In [24]:
# Left as an extra exercise for the reader
from llama_stack_client import Agent

rag_agent = Agent(
    client, 
    model=model_name,
    instructions="You are a helpful assistant",
    tools = [
        {
          "name": "builtin::rag/knowledge_search",
          "args" : {
            "vector_db_ids": [vector_db_id],
          }
        }
    ],
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Arag%2Fknowledge_search "HTTP/1.1 200 OK"


In [25]:
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
user_prompts = [
    "What is Lora?"
]
session_id = rag_agent.create_session(f"rag session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = rag_agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/0e7bbf74-c3bc-4f7a-9b35-789eb7d4095c/session "HTTP/1.1 200 OK"


User> What is Lora?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/0e7bbf74-c3bc-4f7a-9b35-789eb7d4095c/session/606fcb14-b6b8-4c0c-b4cf-ace08c607135/turn "HTTP/1.1 200 OK"


inference> [knowledge_search(query="Lora")]
tool_execution> Tool:knowledge_search Args:{'query': 'Lora'}
tool_execution> Tool:knowledge_search Response:[TextContentItem(text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n', type='text'), TextContentItem(text="Result 1\nContent: .\n\n*Sounds great! How do I use it?*\n\nYou can finetune using any of our recipes with the ``lora_`` prefix, e.g. :ref:`lora_finetune_single_device<lora_finetune_recipe_label>`. These recipes utilize\nLoRA-enabled model builders, which we support for all our models, and also use the ``lora_`` prefix, e.g.\nthe :func:`torchtune.models.llama3.llama3` model has a corresponding :func:`torchtune.models.llama3.lora_llama3`.\nWe aim to provide a comprehensive set of configurations to allow you to get started with training with LoRA quickly,\njust specify any config with ``_lora`` in its name, e.g:\n\n.. code-block:: bash\n\n  tune run lora_finetune_single_device --config llama3/8B_lor